In [ ]:
import copy

from datetime import timedelta, datetime
import imageio
import matplotlib.pyplot as plt
from matplotlib import cm
import multiprocessing
import numpy as np
import os
from pathlib import Path
import pydicom
import pytest
import scipy.ndimage
import scipy.ndimage as ndimage
from scipy.ndimage.interpolation import zoom
from skimage import measure, morphology, segmentation
from time import time, sleep
from tqdm import trange, tqdm
import torch                                                                                           
import torch.nn as nn
import torch.nn.functional as F                                   
from torch.utils.data import Dataset, random_split, DistributedSampler, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import warnings                        
import pandas as pd

import os
import copy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import pydicom
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from torch.utils.data import Dataset
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange
from time import time
import warnings
from scipy.ndimage.interpolation import zoom
from enum import Enum
from torchvision import transforms
from skimage.measure import label, regionprops
from skimage.segmentation import clear_border
import pytest
from torch.optim.lr_scheduler import StepLR

from joblib import Parallel, delayed
import gc

In [ ]:
root_dir = '/kaggle/input/osic-pulmonary-fibrosis-progression/'
# root_dir="/kaggle/input/osic-pulmonary-fibrosis-progression/"
test_dir = '/kaggle/input/osic-pulmonary-fibrosis-progression/test/'
model_dir = '/kaggle/working/'
# model_file = '../input/latentspace-weights/diophantus.pt'
resize_dims = (40, 256, 256)
clip_bounds = (-1000, 200)
watershed_iterations = 1
pre_calculated_mean = 0.02865046213070556
latent_features = 10
batch_size = 32
learning_rate = 3e-03
num_epochs = 150
val_size = 0.2
tensorboard_dir = '/kaggle/working/runs/'

pretrained_weigths_dir = '/kaggle/input/latentspace-weights/'
pretrained_ae_weigths = os.path.join(pretrained_weigths_dir,'diophantus.pt')
cache_dir = '/kaggle/input/osic-preprocrsseddata'
latent_dir = '/kaggle/working/latent'

os.mkdir(os.path.join(model_dir,'latent'))

test_cache_dir="kaggle/working/test_cache"
os.mkdir(os.path.join(model_dir,'test_cache'))

test_latent_dir="kaggle/working/test_latent"
os.mkdir(os.path.join(model_dir,'test_latent'))
test_size=0.2
batch_size = 32
learning_rate = 3e-03
num_epochs =25
quantiles = (0.2, 0.5, 0.8)
model_name ='cauchy'

num_kfolds = 5
# batch_size = 32
# learning_rate = 3e-03
# num_epochs = 1000
es_patience = 20

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import pydicom
from fastai2.basics           import *
from fastai2.medical.imaging  import *


In [ ]:
# df=pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
# df

In [ ]:
# df.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])

In [ ]:
# df

In [ ]:
import pickle
class ClinicalDataset(Dataset):
    def __init__(self, root_dir, ctscans_dir, mode, transform=None,
                 cache_dir=None):
        self.transform = transform
        self.mode = mode
        self.ctscans_dir = ctscans_dir
        self.cache_dir = None if cache_dir is None else cache_dir

        # If cache_dir is set, use cached values...
        if cache_dir is not None:
            self.raw = pd.read_csv(os.path.join(self.cache_dir,f'tabular_{self.mode}.csv'))
#             self.raw=self.raw.drop(self.raw.loc[self.raw['Patient']=='ID00108637202209619669361']
#                             .index).reset_index(drop=True)
            with open(os.path.join(self.cache_dir,'features_list.pkl'), "rb") as fp:
                self.FE = pickle.load(fp)
            return

        # ...otherwise, pre-process
        tr = pd.read_csv(os.path.join(root_dir,"train.csv"))
#         tr=tr.drop(tr.loc[tr['Patient']=='ID00108637202209619669361']
#                             .index).reset_index(drop=True)
        tr.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])
        chunk = pd.read_csv(os.path.join(root_dir,"test.csv"))

        sub = pd.read_csv(os.path.join(root_dir,"sample_submission.csv"))
        sub['Patient'] = sub['Patient_Week'].apply(lambda x: x.split('_')[0])
        sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
        sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
        sub = sub.merge(chunk.drop('Weeks', axis=1), on="Patient")

        tr['WHERE'] = 'train'
        chunk['WHERE'] = 'val'
        sub['WHERE'] = 'test'
        data = tr.append([chunk, sub])

        data['min_week'] = data['Weeks']
        data.loc[data.WHERE == 'test', 'min_week'] = np.nan
        data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

        base = data.loc[data.Weeks == data.min_week]
        base = base[['Patient', 'FVC']].copy()
        base.columns = ['Patient', 'min_FVC']
        base['nb'] = 1
        base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
        base = base[base.nb == 1]
        base.drop('nb', axis=1, inplace=True)

        data = data.merge(base, on='Patient', how='left')
        data['base_week'] = data['Weeks'] - data['min_week']
        del base
        gc.collect()
        
        COLS = ['Sex', 'SmokingStatus']
        self.FE = []
        for col in COLS:
            for mod in data[col].unique():
                self.FE.append(mod)
                data[mod] = (data[col] == mod).astype(int)

        data['age'] = (data['Age'] - data['Age'].min()) / \
                      (data['Age'].max() - data['Age'].min())
        data['BASE'] = (data['min_FVC'] - data['min_FVC'].min()) / \
                       (data['min_FVC'].max() - data['min_FVC'].min())
        data['week'] = (data['base_week'] - data['base_week'].min()) / \
                       (data['base_week'].max() - data['base_week'].min())
        data['percent'] = (data['Percent'] - data['Percent'].min()) / \
                          (data['Percent'].max() - data['Percent'].min())
        self.FE += ['age', 'percent', 'week', 'BASE']

        self.raw = data.loc[data.WHERE == mode].reset_index()
#         print(len(data))
        del data

    def __len__(self):
        return len(self.raw)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        patient_id = self.raw['Patient'].iloc[idx]
        patient_path = os.path.join(self.ctscans_dir,patient_id)
        if self.cache_dir is None:
            patient_path = os.path.join(self.ctscans_dir,patient_id)
            image, slices = self.load_scan(patient_path)
            

            image,metadata=self.resample(image,slices)
           
        else:
            file_path=os.path.join(self.cache_dir,f'{patient_id}.pt')
            try:
                image = torch.load(file_path)
            except:
                image=torch.load("/kaggle/input/osic-preprocrsseddata/ID00078637202199415319443.pt")
            meta=os.listdir(patient_path)
            meta0=meta[0]
#             
            metadata = pydicom.read_file(os.path.join(patient_path,meta0))
           
        sample = {
            'features': self.raw[self.FE].iloc[idx].values,
            'image': image,
            'metadata': metadata,
            'target': self.raw['FVC'].iloc[idx]
        }
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def parallel_save(self,f,path,bar):
        sample=self[f]
        patient_id = sample['metadata'].PatientID
        sample['metadata'].save_as(os.path.join(cache_dir,f'{patient_id}.dcm'))
      
    
    def cache(self, cache_dir):
        Path(cache_dir).mkdir(exist_ok=True, parents=True)

        # Cache raw features table
        self.raw.to_csv(os.path.join(cache_dir,f'tabular_{self.mode}.csv'), index=False)

        # Cache features list
        with open(os.path.join(cache_dir,'features_list.pkl'), "wb") as fp:
            pickle.dump(self.FE, fp)

        # Cache images and metadata
        self.raw['index'] = self.raw.index
        idx_unique = self.raw.groupby('Patient').first()['index'].values
        bar = tqdm(idx_unique.tolist())
        for idx in bar:
            sample = self[idx]
            patient_id = sample['metadata'].PatientID


            if sample['metadata'].TotalSlices>=1000 and self.mode=="test":
                im=torch.load("/kaggle/input/osic-preprocrsseddata/ID00078637202199415319443.pt")
                torch.save(im, Path(cache_dir)/f'{patient_id}.pt')
            else:  
                torch.save(sample['image'], os.path.join(cache_dir,f'{patient_id}.pt'))
#             sample['metadata'].save_as(os.path.join(cache_dir,f'{patient_id}.dcm'))
            
    @staticmethod
    def load_scan(path):
        
#        
        slices = [pydicom.dcmread(os.path.join(path,p)) for p in os.listdir(path)]
#             slices = [pydicom.dcmread(dcm_path + "/" + file) for file in listdir(dcm_path)]
#         print(slices[0].PatientID)
#        print("load_scan")
#         if slices[0].PatientID=="ID00132637202222178761324":
#             slice_thickness=0.625
        
#         elif slices[0].PatientID=="ID00128637202219474716089":
#             slice_thickness=5.0
            
#         elif slices[0].PatientID=="ID00173637202238329754031":
#             slice_thickness=1.0
            
#         elif (slices[0].PatientID=="ID00052637202186188008618") or (slices[0].PatientID=="ID00011637202177653955184"):
#             return slices[0],slices
            
        
#         else:
           
        try:
            slices.sort(key = lambda x: float(x.InstanceNumber))
#             slice_thickness=np.abs(slices[0].SpacingBetweenSlices)
#             slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))

        except AttributeError:
            warnings.warn(f'Patient {slices[0].PatientID} CT scan does not '
                      f'have "ImagePositionPatient". Assuming filenames '
                      f'in the right scan order.')
#             try:'ImagePositionPatient' in slices[0].dir():
#             try:
#             slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
#                 slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
#             except:
#                 slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        arr=[]
        for s in slices:
            if s.Rows!=s.Columns:
                s_data=s.pixel_array
                s_cropped = s_data[~np.all(s_data == 0, axis=1)]
                s_cropped = s_cropped[:, ~np.all(s_cropped == 0, axis=0)]
                new_pix_array=s_cropped
                s.PixelData = new_pix_array.tostring()
                (s.Rows,s.Columns)=new_pix_array.shape

            arr.append(s.pixel_array)
#             s.SliceThickness = slice_thickness
            
        
        
        
        image=np.stack(arr)
        image=image.astype(float)
        
#         image = np.stack([s.pixel_array.astype(float) for s in slices])
        return image, slices
    
    @staticmethod
    def resample(image, scan,new_spacing=[1,1,1]):
#         print("inresample")
#         print(scan[0].PatientID)
      # print(type(scan))
#         print(scan[0].SliceThickness)
        
        
#         spacing=np.array([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]], dtype=np.float32)
#         resize_factor = spacing / new_spacing
#         new_real_shape = image.shape * resize_factor
#         new_shape = np.round(new_real_shape)
#         real_resize_factor = new_shape / image.shape
#         new_spacing = spacing / real_resize_factor   
# #         print("before zoom resizr") 
#         image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
#         print("after zoom resie")
        metadata=scan[0]
    
        metadata.TotalSlices=len(scan[0])
# #         print(metadata)
# #         print(type(metadata))
#         print(metadata.TotalSlices)
        return image, scan[0]
            



**Preprocessing**

In [ ]:
class CropBoundingBox:
    @staticmethod
    def bounding_box(img3d: np.array):
        mid_img = img3d[int(img3d.shape[0] / 2)]
        same_first_row = (mid_img[0, :] == mid_img[0, 0]).all()
        same_first_col = (mid_img[:, 0] == mid_img[0, 0]).all()
        if same_first_col and same_first_row:
            return True
        else:
            return False

    def __call__(self, sample):
        features,image,data,target = sample['features'],sample['image'], sample['metadata'],sample['target']
        
        if not self.bounding_box(image):
            return sample

        mid_img = image[int(image.shape[0] / 2)]
        r_min, r_max = None, None
        c_min, c_max = None, None
        for row in range(mid_img.shape[0]):
            if not (mid_img[row, :] == mid_img[0, 0]).all() and r_min is None:
                r_min = row
            if (mid_img[row, :] == mid_img[0, 0]).all() and r_max is None \
                    and r_min is not None:
                r_max = row
                break

        for col in range(mid_img.shape[1]):
            if not (mid_img[:, col] == mid_img[0, 0]).all() and c_min is None:
                c_min = col
            if (mid_img[:, col] == mid_img[0, 0]).all() and c_max is None \
                    and c_min is not None:
                c_max = col
                break

        image = image[:, r_min:r_max, c_min:c_max]
        return {'features':features,'image': image, 'metadata': data,'target':target}

### 3.2.2. convert_to_hu.py


In [ ]:
class ConvertToHU:
    def __call__(self, sample):
        features,image,data,target = sample['features'],sample['image'], sample['metadata'],sample['target']

        img_type = data.ImageType
        is_hu = img_type[0] == 'ORIGINAL' and not (img_type[2] == 'LOCALIZER')
        # if not is_hu:
        #     warnings.warn(f'Patient {data.PatientID} CT Scan not cannot be'
        #                   f'converted to Hounsfield Units (HU).')

        intercept = data.RescaleIntercept
        slope = data.RescaleSlope
        image = (image * slope + intercept).astype(np.int16)
        return {'features':features,'image': image, 'metadata': data,'target':target}

In [ ]:
class Resize:
    def __init__(self, output_size):
        assert isinstance(output_size, tuple)
        self.output_size = output_size

    def __call__(self, sample):
        features,image,data,target = sample['features'],sample['image'], sample['metadata'],sample['target']
        resize_factor = np.array(self.output_size) / np.array(image.shape)
        image = zoom(image, resize_factor, mode='nearest')
        return {'features':features,'image': image, 'metadata': data,'target':target}

In [ ]:
class Clip:
    def __init__(self, bounds=(-1000, 500)):
        self.min = min(bounds)
        self.max = max(bounds)

    def __call__(self, sample):
        features,image,data,target = sample['features'],sample['image'], sample['metadata'],sample['target']
        image[image < self.min] = self.min
        image[image > self.max] = self.max
        return {'features':features,'image': image, 'metadata': data,'target':target}

Segmentation

In [ ]:
class MaskWatershed:
    def __init__(self, min_hu, iterations, show_tqdm):
        self.min_hu = min_hu
        self.iterations = iterations
        self.show_tqdm = show_tqdm

    def __call__(self, sample):
        features,image,data,target = sample['features'],sample['image'], sample['metadata'],sample['target']

        stack = []
        if self.show_tqdm:
            bar = trange(image.shape[0])
            bar.set_description(f'Masking CT scan {data.PatientID}')
        else:
            bar = range(image.shape[0])
        for slice_idx in bar:
            sliced = image[slice_idx]
            stack.append(self.seperate_lungs(sliced, self.min_hu,
                                             self.iterations))

        return {
            'features':features,
            'image': np.stack(stack),
            'metadata': sample['metadata'],
            'target':target
        }

    @staticmethod
    def seperate_lungs(image, min_hu, iterations):
        h, w = image.shape[0], image.shape[1]

        marker_internal, marker_external, marker_watershed = MaskWatershed.generate_markers(image)

        # Sobel-Gradient
        sobel_filtered_dx = ndimage.sobel(image, 1)
        sobel_filtered_dy = ndimage.sobel(image, 0)
        sobel_gradient = np.hypot(sobel_filtered_dx, sobel_filtered_dy)
        sobel_gradient *= 255.0 / np.max(sobel_gradient)

        watershed = morphology.watershed(sobel_gradient, marker_watershed)

        outline = ndimage.morphological_gradient(watershed, size=(3,3))
        outline = outline.astype(bool)

        # Structuring element used for the filter
        blackhat_struct = [[0, 0, 1, 1, 1, 0, 0],
                           [0, 1, 1, 1, 1, 1, 0],
                           [1, 1, 1, 1, 1, 1, 1],
                           [1, 1, 1, 1, 1, 1, 1],
                           [1, 1, 1, 1, 1, 1, 1],
                           [0, 1, 1, 1, 1, 1, 0],
                           [0, 0, 1, 1, 1, 0, 0]]

        blackhat_struct = ndimage.iterate_structure(blackhat_struct, iterations)

        # Perform Black Top-hat filter
        outline += ndimage.black_tophat(outline, structure=blackhat_struct)

        lungfilter = np.bitwise_or(marker_internal, outline)
        lungfilter = ndimage.morphology.binary_closing(lungfilter, structure=np.ones((5,5)), iterations=3)

        segmented = np.where(lungfilter == 1, image, min_hu * np.ones((h, w)))

        return segmented  #, lungfilter, outline, watershed, sobel_gradient

    @staticmethod
    def generate_markers(image, threshold=-400):
        h, w = image.shape[0], image.shape[1]

        marker_internal = image < threshold
        marker_internal = segmentation.clear_border(marker_internal)
        marker_internal_labels = measure.label(marker_internal)

        areas = [r.area for r in measure.regionprops(marker_internal_labels)]
        areas.sort()

        if len(areas) > 2:
            for region in measure.regionprops(marker_internal_labels):
                if region.area < areas[-2]:
                    for coordinates in region.coords:
                        marker_internal_labels[coordinates[0], coordinates[1]] = 0

        marker_internal = marker_internal_labels > 0

        # Creation of the External Marker
        external_a = ndimage.binary_dilation(marker_internal, iterations=10)
        external_b = ndimage.binary_dilation(marker_internal, iterations=55)
        marker_external = external_b ^ external_a

        # Creation of the Watershed Marker
        marker_watershed = np.zeros((h, w), dtype=np.int)
        marker_watershed += marker_internal * 255
        marker_watershed += marker_external * 128

        return marker_internal, marker_external, marker_watershed

In [ ]:
class Normalize:
    def __init__(self, bounds=(-1000, 500)):
        self.min = min(bounds)
        self.max = max(bounds)

    def __call__(self, sample):
        features,image,data,target = sample['features'],sample['image'], sample['metadata'],sample['target']
        image = image.astype(np.float)
        image = (image - self.min) / (self.max - self.min)
        return {'features':features,'image': image, 'metadata': data,'target':target}
    

class ToTensor:
    def __init__(self, add_channel=True):
        self.add_channel = add_channel

    def __call__(self, sample):
        features,image,data,target = sample['features'],sample['image'], sample['metadata'],sample['target']
        if self.add_channel:
            image = np.expand_dims(image, axis=0)

        return {'features':features,'image': torch.from_numpy(image), 'metadata': data,'target':target}
    
class ZeroCenter:
    def __init__(self, pre_calculated_mean):
        self.pre_calculated_mean = pre_calculated_mean

    def __call__(self, sample):
        return {
            'features': sample['features'],
            'image': sample['image'] - self.pre_calculated_mean,
            'metadata': sample['metadata'],
            'target': sample['target']
        }

In [ ]:
def show(list_imgs, cmap=cm.bone):
    list_slices = []
    for img3d in list_imgs:
        slc = int(img3d.shape[0] / 2)
        img = img3d[slc]
        list_slices.append(img)
    
    fig, axs = plt.subplots(1, 5, figsize=(15, 7))
    for i, img in enumerate(list_slices):
        axs[i].imshow(img, cmap=cmap)
        axs[i].axis('off')
        
    plt.show()

In [ ]:
data = ClinicalDataset(
    root_dir=root_dir,
    ctscans_dir=os.path.join(root_dir,'test'),
    mode='test',
    transform=transforms.Compose([
        CropBoundingBox(),
        ConvertToHU(),
        Resize((40, 256, 256)),
        Clip(bounds=clip_bounds),
        MaskWatershed(
            min_hu=min(clip_bounds),
            iterations=watershed_iterations,
            show_tqdm=False),
        Normalize(bounds=clip_bounds),
        ToTensor(),
         ZeroCenter(pre_calculated_mean=pre_calculated_mean)
    ]))

data.cache("/kaggle/working/test_cache")                         

In [ ]:

def check_submit():
    tr = pd.read_csv(Path(root_dir)/"train.csv")
    tr.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])
    chunk = pd.read_csv(Path(root_dir)/"test.csv")

    sub = pd.read_csv(Path(root_dir)/"sample_submission.csv")
    sub['Patient'] = sub['Patient_Week'].apply(lambda x: x.split('_')[0])
    sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
    sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
    sub = sub.merge(chunk.drop('Weeks', axis=1), on="Patient")

    tr['WHERE'] = 'train'
    chunk['WHERE'] = 'val'
    sub['WHERE'] = 'test'
    data = tr.append([chunk, sub])

    data['min_week'] = data['Weeks']
    data.loc[data.WHERE == 'test', 'min_week'] = np.nan
    data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

    base = data.loc[data.Weeks == data.min_week]
    base = base[['Patient', 'FVC']].copy()
    base.columns = ['Patient', 'min_FVC']
    base['nb'] = 1
    base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
    base = base[base.nb == 1]
    base.drop('nb', axis=1, inplace=True)

    data = data.merge(base, on='Patient', how='left')
    data['base_week'] = data['Weeks'] - data['min_week']
    del base

    COLS = ['Sex', 'SmokingStatus']
    FE = []
    for col in COLS:
        for mod in data[col].unique():
            FE.append(mod)
            data[mod] = (data[col] == mod).astype(int)

    data['age'] = (data['Age'] - data['Age'].min()) / \
                  (data['Age'].max() - data['Age'].min())
    data['BASE'] = (data['min_FVC'] - data['min_FVC'].min()) / \
                   (data['min_FVC'].max() - data['min_FVC'].min())
    data['week'] = (data['base_week'] - data['base_week'].min()) / \
                   (data['base_week'].max() - data['base_week'].min())
    data['percent'] = (data['Percent'] - data['Percent'].min()) / \
                      (data['Percent'].max() - data['Percent'].min())
    FE += ['age', 'percent', 'week', 'BASE']
    raw = data.loc[data.WHERE == 'test'].reset_index()
    del data
    return raw


In [ ]:
dm = ClinicalDataset(
    root_dir=root_dir,
    ctscans_dir=os.path.join(root_dir,'test'),
    mode='test',
    transform=transforms.Compose([
        CropBoundingBox(),
        ConvertToHU(),
        Resize((40, 256, 256)),
        Clip(bounds=clip_bounds),
        MaskWatershed(
            min_hu=min(clip_bounds),
            iterations=watershed_iterations,
            show_tqdm=False),
        Normalize(bounds=clip_bounds),
        ToTensor(),
         ZeroCenter(pre_calculated_mean=pre_calculated_mean)
    ]))


QuantModel

In [ ]:
class QuantModel(nn.Module):
    def __init__(self, in_tabular_features=9, in_ctscan_features=10,
                 out_quantiles=3):
        super(QuantModel, self).__init__()
        # This line is new. We need to know a priori the number
        # of latent features to properly flatten the tensor
        self.in_ctscan_features = in_ctscan_features

        self.fc1 = nn.Linear(in_tabular_features, 512)
        self.fc=nn.Linear(512,256)
        self.fc2 = nn.Linear(in_ctscan_features, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fcl=nn.Linear(256,128)
        self.fc4 = nn.Linear(128, out_quantiles)
#         self.dropout = nn.Dropout(p=0.5) 
    def forward(self, x1, x2):
        # Now the quant model has 2 inputs: x1 (the tabular features)
        # and x2 (the pre-computed latent features)
#         print(x1.shape)
        x1 = F.relu(self.fc1(x1))
#         x1=self.dropout(x1)
        x1=F.relu(self.fc(x1))
#         print("x1")
#         print(x1.shape)
#         print("x2")
#         print(x2.shape)
        # Flattens the latent features and concatenate with tabular features
#         x2 = x2.view(-1, self.in_ctscan_features)
#         print(x2.shape)
        x2 = F.relu(self.fc2(x2))
        x2=F.relu(self.fc(x2))
#         print(x2.shape)
        x = torch.cat([x1, x2], dim=1)
#         x=self.dropout(x)
        x = F.relu(self.fc3(x))
        x=F.relu(self.fcl(x))
        x = self.fc4(x)
        return x

Quant loss

In [ ]:
def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []                                              
    for i, q in enumerate(quantiles):                                                                
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))   
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))                                                
    return loss

AutoEncoder

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, latent_features=latent_features):
        super(AutoEncoder, self).__init__()
        # Encoder
        self.conv1 = nn.Conv3d(1, 16, 3)
        self.conv2 = nn.Conv3d(16, 32, 3)
        self.conv3 = nn.Conv3d(32, 96, 2)
        self.conv4 = nn.Conv3d(96, 1, 1)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
        self.pool2 = nn.MaxPool3d(kernel_size=3, stride=3, return_indices=True)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
        self.pool4 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
        self.fc1 = nn.Linear(10 * 10, latent_features)
        # Decoder
        self.fc2 = nn.Linear(latent_features, 10 * 10)
        self.deconv0 = nn.ConvTranspose3d(1, 96, 1)
        self.deconv1 = nn.ConvTranspose3d(96, 32, 2)
        self.deconv2 = nn.ConvTranspose3d(32, 16, 3)
        self.deconv3 = nn.ConvTranspose3d(16, 1, 3)
        self.unpool0 = nn.MaxUnpool3d(kernel_size=2, stride=2)
        self.unpool1 = nn.MaxUnpool3d(kernel_size=2, stride=2)
        self.unpool2 = nn.MaxUnpool3d(kernel_size=3, stride=3)
        self.unpool3 = nn.MaxUnpool3d(kernel_size=2, stride=2)

    def encode(self, x, return_partials=True):
        # Encoder
        x = self.conv1(x)
        up3out_shape = x.shape
        x, i1 = self.pool1(x)

        x = self.conv2(x)
        up2out_shape = x.shape
        x, i2 = self.pool2(x)

        x = self.conv3(x)
        up1out_shape = x.shape
        x, i3 = self.pool3(x)

        x = self.conv4(x)
        up0out_shape = x.shape
        x, i4 = self.pool4(x)

        x = x.view(-1, 10 * 10)
        x = F.relu(self.fc1(x))

        if return_partials:
            return x, up3out_shape, i1, up2out_shape, i2, up1out_shape, i3, \
                   up0out_shape, i4

        else:
            return x

    def forward(self, x):
        x, up3out_shape, i1, up2out_shape, i2, \
        up1out_shape, i3, up0out_shape, i4 = self.encode(x)

        # Decoder
        x = F.relu(self.fc2(x))
        x = x.view(-1, 1, 1, 10, 10)
        x = self.unpool0(x, output_size=up0out_shape, indices=i4)
        x = self.deconv0(x)
        x = self.unpool1(x, output_size=up1out_shape, indices=i3)
        x = self.deconv1(x)
        x = self.unpool2(x, output_size=up2out_shape, indices=i2)
        x = self.deconv2(x)
        x = self.unpool3(x, output_size=up3out_shape, indices=i1)
        x = self.deconv3(x)

        return x

Cache all pre-processed 3D CT Scans and pre-compute all latent features

In [ ]:
# Helper function that generates all latent features
class GenerateLatentFeatures:
    def __init__(self, autoencoder,cache_dir, latent_dir,mode):
        self.autoencoder = autoencoder
        self.latent_dir =latent_dir
        self.cache_dir = cache_dir
        self.mode=mode

    def __call__(self, sample):
        patient_id = sample['metadata'].PatientID
        cached_latent_file = os.path.join(self.latent_dir,f'{patient_id}_lat.pt')
        if os.path.isfile(cached_latent_file):
            latent_features = torch.load(cached_latent_file)
        else:
            with torch.no_grad():
                img=sample['image'].to(device)
                img = img.float().unsqueeze(0)
                latent_features = self.autoencoder.encode(
                    img, return_partials=False).squeeze(0)
            torch.save(latent_features, cached_latent_file)
            if self.mode=='test':
#                 print(os.path.join(self.cache,f'{patient_id}.pt'))
#                 pathlib.Path.unlink(self.cache_dir/f'{patient_id}.pt')
                try:
                    os.remove(os.path.join(self.cache_dir,f'{patient_id}.pt'))
                except AttributeError:
                    warnings.warn(f'Patient {slices[0].PatientID} CT scan does not '
                      f'have "ImagePositionPatient". Assuming filenames '
                      f'in the right scan order.')

        return {
            'tabular_features': sample['features'],
            'latent_features': latent_features,
            'target': sample['target']
        }

In [ ]:
autoencoder = AutoEncoder()
autoencoder.load_state_dict(torch.load(
    pretrained_ae_weigths,
    map_location=torch.device('cuda')
))
# print(pretrained_ae_weigths.shape)
autoencoder.to(device)
autoencoder.eval()

data = ClinicalDataset(
    root_dir=root_dir,
    ctscans_dir=os.path.join(root_dir,'train'),
    cache_dir=cache_dir,
    mode='train',
    transform=GenerateLatentFeatures(autoencoder,cache_dir, latent_dir,'train')
)
for i in trange(len(data)):
    sample = data[i]
#     print(sample['latent_features'].shape)
#     assert sample['latent_features'].shape == (96, 2, 20, 20)


* 1. Overfit a small batch before moving forward

In [ ]:
dataloader = DataLoader(data, batch_size=batch_size,
                        shuffle=True, num_workers=0)
# dataloader=dataloader.cuda()num_workers to 0 and pin_memory to False.
batch = next(iter(dataloader))

model = QuantModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

bar = trange(50)
for epoch in bar:
    inputs1 = batch['tabular_features'].float().to(device)
    inputs2 = batch['latent_features'].float().to(device)
    targets = batch['target'].to(device)

    optimizer.zero_grad()
    preds = model(inputs1, inputs2)
    loss = quantile_loss(preds, targets, quantiles)
    loss.backward()

    optimizer.step()
    

    bar.set_postfix(loss=f'{loss.item():0.1f}')

In [ ]:
# Helper generator that group splits
def group_split(dataset, groups, test_size=0.2):
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size)
    idx = list(gss.split(dataset.raw, dataset.raw, groups))
    train = Subset(dataset, idx[0][0])
    val = Subset(dataset, idx[0][1])
    return train, val

def group_kfold(dataset,n_splits):
    gkf = GroupKFold(n_splits=n_splits)
    groups = dataset.raw['Patient']
    for train_idx, val_idx in gkf.split(dataset.raw, dataset.raw, groups):
        train = Subset(dataset, train_idx)
        val = Subset(dataset, val_idx)
        yield train, val


# Helper function with competition metric
def metric(preds, targets):
#     print(preds.shape)
#     print(targets.shape)
    
    sigma = preds[:, 2] - preds[:, 0]
    sigma[sigma < 70] = 70
    delta = (preds[:, 1] - targets).abs()
    delta[delta > 1000] = 1000
#     print("end")
    return -np.sqrt(2) * delta / sigma - torch.log(np.sqrt(2) * sigma)

# **lr_scheduler method and callbacks**

In [ ]:
trainset, valset = group_split(data, data.raw['Patient'], test_size)

In [ ]:
 dataloaders = {
        'train': DataLoader(trainset, batch_size=batch_size,
                            shuffle=True, num_workers=0),
        'val': DataLoader(valset, batch_size=batch_size,
                          shuffle=False, num_workers=0)
    }

In [ ]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl):
        self.train_dl,self.valid_dl = train_dl,valid_dl
        
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [ ]:
data = DataBunch(dataloaders['train'],dataloaders['val'])

In [ ]:
def get_model(data):
    model = QuantModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer

class Learner():
    def __init__(self, model, opt, data):
        self.model,self.opt,self.data = model,opt,data

In [ ]:
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
    _order=0
    def set_runner(self,run):self.run=run

    def __getattr__(self,k):return getattr(self.run, k)

    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')
    
    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False


class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

In [ ]:
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

In [ ]:
# Set global tracking variables
epoch_loss = {'train': np.inf, 'val': np.inf}
epoch_metric = {'train': -np.inf, 'val': -np.inf}
best_loss = np.inf
best_model_wts = None


model_dir = '/kaggle/working/'
Path(model_dir).mkdir(parents=True, exist_ok=True)
now = datetime.now()
fname = f'{model_name}-{now.year}{now.month:02d}{now.day:02d}.pth'
model_file = Path(model_dir) / fname

In [ ]:
class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        self.in_train = False
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs
        self.best_loss = np.inf
        self.val_loss = np.inf
        self.best_model_wts = None
#         self.bar=torch.tensor(0.)

    @property
    def opt(self):       return self.learn.opt
    @property
    def model(self):     return self.learn.model
#     @property
#     def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data
    
    @property
    def best_model_weights(self):
        return self.best_model_wts
    
    def one_batch(self, inputs1, inputs2,targets):
        try:
            self.inputs1,self.inputs1,self.targets = inputs1,inputs2,targets
            self.loss =torch.tensor(0.)
            self('begin_batch')
            self.preds = self.model(inputs1, inputs2)
    #         model(inputs1, inputs2)
            self('after_pred')
            self.loss = quantile_loss(self.preds, self.targets, quantiles)
            self('after_loss')
            if not self.in_train: return
            self.loss.backward()
            self('after_backward')
            self.opt.step()
            self('after_step')
            self.opt.zero_grad()
        
        except CancelBatchException: self('after_cancel_batch')
            
        finally: self('after_batch')


    def all_batches(self, dl):
        try:
            self.iters = len(dl)
            self.bar = tqdm(dl)
            self.mode=None
            for batch in self.bar:
                if self.stop: break
                if self.in_train: mode='train'
                else: mode='valid'
                self.bar.set_description(f'Epoch {self.epoch} {mode}'.ljust(20))    
                inputs1 = batch['tabular_features'].float().to(device)
                inputs2 = batch['latent_features'].float().to(device)
                targets = batch['target'].to(device)
            
                self.one_batch(inputs1,inputs2,targets)
#             self('after_batch')
#         self.stop=False
        
        except CancelEpochException: self('after_cancel_epoch')

    def fit(self, epochs, learn):
        self.epochs,self.learn= epochs,learn


        try:
            for cb in self.cbs: cb.set_runner(self)
            self('begin_fit')
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad(): 
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                self('after_epoch')
        
        except CancelTrainException: self('after_cancel_train')
            
        finally:
            self('after_fit')
            self.learn = None

    def __call__(self, cb_name):
        res = False
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) or res
        return res

In [ ]:
class AvgStats():
    
    def __init__(self, metrics, in_train): 
        self.metrics,self.in_train = listify(metrics),in_train

    
    def reset(self):
        self.tot_loss,self.count = 0.,0
        self.tot_mets = [0.] * len(self.metrics)
        
    @property
    def all_stats(self): return [self.tot_loss] + self.tot_mets
    @property
    def avg_stats(self):
#         print(self.all_stats)
        return [o/self.count for o in self.all_stats]
    @property
    def best_weights(self):
        return self.best_model_wts 
        
    def saved_weights(self,run):
#         if not self.count: return ""
        if self.in_train==False:
            run.val_loss=self.avg_stats[0]
        if (self.in_train==False) and (self.avg_stats[0] < run.best_loss):
            run.best_loss = self.avg_stats[0]
            run.best_model_wts = copy.deepcopy(run.model.state_dict())
            torch.save(run.best_model_wts, model_file)       
            
#         return f"{'train' if self.in_train else 'valid'}: loss:{self.avg_stats[0]:0.1f}, metric:{self.avg_stats[1]:0.4f}"

    def accumulate(self, run):
        self.batch_mat=[0.]
        bn = run.inputs1.size(0)
#         bn = run.xb.shape[0]
        
#         self.running_loss += loss.item() * inputs1.size(0)
        self.tot_loss += run.loss.item() * bn
#         self.running_metric+=metric(self.preds, self.targets).sum()
#         running_loss += loss.item() * inputs1.size(0)
#         running_metric += metric(preds, targets).sum()
        self.count += bn
#         print(self.metrics)
        for i,m in enumerate(self.metrics):
            k=m(run.preds, run.targets).sum()
            self.tot_mets[i] += k


        run.bar.set_postfix(loss=f'{self.tot_loss / self.count :0.1f}',
                            metric=f'{self.tot_mets[0] / self.count :0.4f}')

class AvgStatsCallback(Callback):
    _order=1
    def __init__(self,metrics):
        self.train_stats,self.valid_stats=AvgStats(metrics,True),AvgStats(metrics,False)
  
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
  
    def after_loss(self):
        stats=self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad():
            stats.accumulate(self.run)
  
    def after_epoch(self):
        self.valid_stats.saved_weights(self.run)


In [ ]:
class Recorder(Callback):
    def begin_fit(self):
        self.lrs,self.losses=[],[]
  
    def after_batch(self):
        if not self.in_train:return
        self.lrs.append(self.opt.param_groups[-1]['lr'])
        self.losses.append(self.loss.detach().cpu())
  
    def plot_lr(self):
        plt.plot(self.lrs)
  
    def plot_losses(self):
        plt.plot(self.losses)

class ParamScheduler(Callback):
    _order=1
    def __init__(self,pname,sched_func):
        self.pname,self.sched_func=pname,sched_func
  
    def set_param(self):
        for pg in self.opt.param_groups:
            pg[self.pname]=self.sched_func(self.n_epochs/self.epochs)
  
    def begin_batch(self):
        if self.in_train : self.set_param()


In [ ]:
class EarlyStopping(Callback):
    _order=2
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
#         self.early_stop = False
#         self.val_loss_min = np.Inf
        self.delta = delta
#         self.path = path


    def after_epoch(self):
        with torch.no_grad():
            
            score = -self.run.val_loss
            if self.best_score is None:
                print("k")
                self.best_score = score
                self.save_checkpoint(self.run.val_loss, self.run.model)
            elif score < self.best_score + self.delta:
                self.counter += 1
                
                if self.counter >= self.patience:
#                     self.early_stop = True
                    raise CancelTrainException()
            else:
                self.best_score = score
                self.save_checkpoint(self.run.val_loss, self.run.model)
                self.counter = 0
        

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), model_file)
        self.val_loss_min = val_loss

In [ ]:
from functools import partial

In [ ]:
#export
def annealer(f):
    def _inner(start, end): return partial(f, start, end)
    return _inner

@annealer
def sched_lin(start, end, pos): return start + pos*(end-start)

In [ ]:
@annealer
def sched_cos(start, end, pos): return start + (1 + math.cos(math.pi*(1-pos))) * (end-start) / 2
@annealer
def sched_no(start, end, pos):  return start
@annealer
def sched_exp(start, end, pos): return start * (end/start) ** pos

def cos_1cycle_anneal(start, high, end):
    return [sched_cos(start, high), sched_cos(high, end)]

torch.Tensor.ndim = property(lambda x: len(x.shape))

In [ ]:
def combine_scheds(pcts, scheds):
    assert sum(pcts) == 1.
    pcts = torch.tensor([0] + listify(pcts))
    assert torch.all(pcts >= 0)
    pcts = torch.cumsum(pcts, 0)
    def _inner(pos):
        idx = (pos >= pcts).nonzero().max()
        if idx == 2: idx = 1
        actual_pos = (pos-pcts[idx]) / (pcts[idx+1]-pcts[idx])
        return scheds[idx](actual_pos)
    return _inner

In [ ]:
from typing import *

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [ ]:
lr_start   = 0.5e-03
lr_max     = 3e-03
lr_min     = 0.1e-03
lr_ramp_percent =  0.3

In [ ]:
sched = combine_scheds([0.4, 0.6], [sched_cos(lr_start, lr_max), sched_cos(lr_max, lr_min)]) 

In [ ]:
learn = Learner(*get_model(data), data)

In [ ]:
# acc_cbf=partial(AvgStatsCallback,metric)
# cbfs=[Recorder,
#       partial(AvgStatsCallback,metric)]
cbfs = [Recorder,
        partial(AvgStatsCallback,metric),
        partial(ParamScheduler, 'lr', sched),
        partial(EarlyStopping,patience=80)]

In [ ]:
run = Runner(cb_funcs=cbfs)

In [ ]:
import math

In [ ]:
run.fit(20, learn)

In [ ]:
run.recorder.plot_lr()

In [ ]:
run.recorder.plot_losses()

In [ ]:
# print(run.best_model_weights)
# a=run.best_model_weights
# # model = QuantModel().to(device)

In [ ]:
model.load_state_dict(run.best_model_wts)
# model.load_state_dict("/kaggle/working/cauchy-20201004.pth")

# print(f'Training complete! Time: {timedelta(seconds=time() - t0)}')
models = [model]

END OF new Approach

# **Training**

In [ ]:
# # Helper generator that group splits
# def group_split(dataset, groups, test_size=0.2):
#     gss = GroupShuffleSplit(n_splits=1, test_size=test_size)
#     idx = list(gss.split(dataset.raw, dataset.raw, groups))
#     train = Subset(dataset, idx[0][0])
#     val = Subset(dataset, idx[0][1])
#     return train, val

# def group_kfold(dataset,n_splits):
#     gkf = GroupKFold(n_splits=n_splits)
#     groups = dataset.raw['Patient']
#     for train_idx, val_idx in gkf.split(dataset.raw, dataset.raw, groups):
#         train = Subset(dataset, train_idx)
#         val = Subset(dataset, val_idx)
#         yield train, val


# # Helper function with competition metric
# def metric(preds, targets):
#     sigma = preds[:, 2] - preds[:, 0]
#     sigma[sigma < 70] = 70
#     delta = (preds[:, 1] - targets).abs()
#     delta[delta > 1000] = 1000
#     return -np.sqrt(2) * delta / sigma - torch.log(np.sqrt(2) * sigma)

In [ ]:
# # Helper class that monitors training
# class Monitor:
#     def __init__(self, model, es_patience, experiment_name, tensorboard_dir,
#                  num_epochs, dataset_sizes, model_file):

#         self.model = model
#         self.model_file = model_file
#         self.es_patience = es_patience
#         self.tensorboard_dir = tensorboard_dir
#         self.dataset_sizes = dataset_sizes
#         date_time = datetime.now().strftime("%Y%m%d-%H%M")
#         log_dir = tensorboard_dir / f'{experiment_name}-{date_time}'
#         self.w = SummaryWriter(log_dir)

#         self.bar = trange(num_epochs, desc=experiment_name)

#         self.epoch_loss = {'train': np.inf, 'val': np.inf}
#         self.epoch_metric = {'train': -np.inf, 'val': -np.inf}
#         self.best_loss = np.inf
#         self.best_model_wts = None

#         self.e = {'train': 0, 'val': 0}  # epoch counter
#         self.t = {'train': 0, 'val': 0}  # global time-step (never resets)
#         self.running_loss = 0.0
#         self.running_metric = 0.0
#         self.es_counter = 0

#     def reset_epoch(self):
#         self.running_loss = 0.0
#         self.running_metric = 0.0

#     def step(self, loss, inputs1,inputs2, preds, targets, phase):
#         self.running_loss += loss.item() * inputs1.size(0)
#         self.running_metric += self.metric(preds, targets).sum()
#         self.t[phase] += 1

#     def log_epoch(self, phase):
#         self.epoch_loss[phase] = self.running_loss / self.dataset_sizes[phase]
#         self.epoch_metric[phase] = self.running_metric / self.dataset_sizes[phase]
#         self.bar.set_postfix(
#             a_train_loss=f'{self.epoch_loss["train"]:0.1f}',
#             b_val_loss=f'{self.epoch_loss["val"]:0.1f}',
#             c_train_metric=f'{self.epoch_metric["train"]:0.4f}',
#             d_val_metric=f'{self.epoch_metric["val"]:0.4f}',
#             es_counter=self.es_counter
#         )
#         self.w.add_scalar(
#             f'Loss/{phase}', self.epoch_loss[phase], self.e[phase])
#         self.w.add_scalar(
#             f'Accuracy/{phase}', self.epoch_metric[phase], self.e[phase])

#         self.e[phase] += 1

#         # Early stop and model backup
#         early_stop = False
#         if phase == 'val':
#             if self.epoch_loss['val'] < self.best_loss:
#                 self.best_loss = self.epoch_loss['val']
#                 self.best_model_wts = copy.deepcopy(self.model.state_dict())
#                 torch.save(self.best_model_wts, self.model_file)
#                 self.es_counter = 0
#             else:
#                 self.es_counter += 1
#                 if self.es_counter >= self.es_patience:
#                     early_stop = True
#                     self.bar.close()

#         return early_stop

#     @staticmethod
#     def metric(preds, targets):
#         sigma = preds[:, 2] - preds[:, 0]
#         sigma[sigma < 70] = 70
#         delta = (preds[:, 1] - targets).abs()
#         delta[delta > 1000] = 1000
#         return -np.sqrt(2) * delta / sigma - torch.log(np.sqrt(2) * sigma)

In [ ]:
# !rm -rf ./logs/ 
# !mkdir ./logs/
# # Download Ngrok to tunnel the tensorboard port to an external port
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# pool = multiprocessing.Pool(processes = 10)
# results_of_processes = [
#     pool.apply_async(os.system, args=(cmd, ), callback=None) for cmd in [
#         f"tensorboard --logdir {tensorboard_dir}/ --host 0.0.0.0 --port 6006 &",
#         "./ngrok http 6006 &"
#     ]
# ]

In [ ]:
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# # Load the data
# autoencoder = AutoEncoder()
# autoencoder.load_state_dict(torch.load(
#     pretrained_ae_weigths,
#     map_location=torch.device('cuda')
# ))
# autoencoder.to(device)

# autoencoder.eval()

# models=[]
# data = ClinicalDataset(
#     root_dir=root_dir,
#     ctscans_dir=os.path.join(root_dir,'train'),
#     cache_dir=cache_dir,
#     mode='train',
#     transform=GenerateLatentFeatures(autoencoder,cache_dir, latent_dir,'train')
# )

# es_patience = 20
# es_counter=0
# early_stop=False

# folds=group_kfold(data,n_splits=5)

# t0 = time()

# for fold, (trainset, valset) in enumerate(folds):

# #     trainset, valset = group_split(data, data.raw['Patient'], test_size)


#     # Prepare to save model weights
#     Path(model_dir).mkdir(parents=True, exist_ok=True)
#     now = datetime.now()
#     fname = f'{model_name}-{now.year}{now.month:02d}{now.day:02d}.pth'
#     model_file = Path(model_dir) / fname

#     dataset_sizes = {'train': len(trainset), 'val': len(valset)}
#     dataloaders = {
#         'train': DataLoader(trainset, batch_size=batch_size,
#                             shuffle=True, num_workers=0),
#         'val': DataLoader(valset, batch_size=batch_size,
#                           shuffle=False, num_workers=0)
#     }

#     # Create the model and optimizer
#     model = QuantModel().to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#     # Set global tracking variables
#     epoch_loss = {'train': np.inf, 'val': np.inf}
#     epoch_metric = {'train': -np.inf, 'val': -np.inf}
#     best_loss = np.inf
#     best_model_wts = None
#     df = pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss'])

#     # Training loop
#     for epoch in range(num_epochs):
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

#             running_loss = 0.0
#             running_metric = 0.0

#             # Iterate over data
#             num_samples = 0
#             bar = tqdm(dataloaders[phase])
#             for batch in bar:
#                 bar.set_description(f'Epoch {epoch} {phase}'.ljust(20))
#                 inputs1 = batch['tabular_features'].float().to(device)
#                 inputs2 = batch['latent_features'].float().to(device)
#                 targets = batch['target'].to(device)

#                 # zero the parameter gradients
#                 optimizer.zero_grad()
#                 # forward
#                 # track gradients if only in train
#                 with torch.set_grad_enabled(phase == 'train'):
#                     preds = model(inputs1, inputs2)
#                     loss = quantile_loss(preds, targets, quantiles)
#                     # backward + optimize only if in training phase
#                     if phase == 'train':
#                         loss.backward()


#                         optimizer.step()

#                 running_loss += loss.item() * inputs1.size(0)
#                 running_metric += metric(preds, targets).sum()

#                 # batch statistics
#                 num_samples += inputs1.size(0)
#                 bar.set_postfix(loss=f'{running_loss / num_samples:0.1f}',
#                                 metric=f'{running_metric / num_samples:0.4f}')

#             # epoch statistics
#             epoch_loss[phase] = running_loss / dataset_sizes[phase]
#             epoch_metric[phase] = running_metric / dataset_sizes[phase]

#             # deep copy the model
#             if phase == 'val' and epoch_loss['val'] < best_loss:
#                 best_loss = epoch_loss['val']
#                 best_model_wts = copy.deepcopy(model.state_dict())
#                 torch.save(best_model_wts, model_file)
#                 es_counter = 0
#             else:
#                 es_counter += 1
#                 if es_counter >= es_patience:
#                     early_stop = True
#                     bar.close()

#         df = df.append({
#             'epoch': epoch + 1,
#             'train_loss': epoch_loss["train"],
#             'val_loss': epoch_loss["val"]
#         }, ignore_index=True)
        
# #         early_stop = log_epoch(data,phase)
        
#         if early_stop:
#             break
#     # Save training statistics
#     # fname = f'{model_name}-{now.year}{now.month:02d}{now.day:02d}.csv'
#     # csv_file = os.path.join(model_dir,fname)
#     # df.to_csv(csv_file, index=Falsex

#     # load best model weights
#     model.load_state_dict(best_model_wts)

#     models.append(model)

# print(f'Training complete! Time: {timedelta(seconds=time() - t0)}')
# # models = [model]

In [ ]:
# models = []

# # Load the data
# # data = ClinicalDataset(root_dir=root_dir, mode='train')
# autoencoder = AutoEncoder()
# autoencoder.load_state_dict(torch.load(
#     pretrained_ae_weigths,
#     map_location=torch.device('cuda')
# ))
# autoencoder.to(device)

# autoencoder.eval()

# models=[]
# data = ClinicalDataset(
#     root_dir=root_dir,
#     ctscans_dir=os.path.join(root_dir,'train'),
#     cache_dir=cache_dir,
#     mode='train',
#     transform=GenerateLatentFeatures(autoencoder,cache_dir, latent_dir,'train')
# )

# folds=group_kfold(data,n_splits=5)
# t0 = time()

# for fold, (trainset, valset) in enumerate(folds):
#     # Prepare to save model weights
#     Path(model_dir).mkdir(parents=True, exist_ok=True)
#     now = datetime.now()
#     fname = f'{model_name}-{now.year}{now.month:02d}{now.day:02d}_{fold}.pth'
#     model_file = Path(model_dir) / fname

#     dataset_sizes = {'train': len(trainset), 'val': len(valset)}
#     dataloaders = {
#         'train': DataLoader(trainset, batch_size=batch_size,
#                             shuffle=True, num_workers=0),
#         'val': DataLoader(valset, batch_size=batch_size,
#                           shuffle=False, num_workers=0)
#     }

#     # Create the model and optimizer
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model = QuantModel().to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
#     monitor = Monitor(
#         model=model,
#         es_patience=es_patience,
#         experiment_name=f'{model_name}_fold_{fold}',
#         tensorboard_dir=Path(tensorboard_dir),
#         num_epochs=num_epochs,
#         dataset_sizes=dataset_sizes,
#         model_file=model_file
#     )

#     # Training loop
#     for epoch in monitor.bar:
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

#             monitor.reset_epoch()

#             # Iterate over data
#             for batch in dataloaders[phase]:
#                 inputs1 = batch['tabular_features'].float().to(device)
#                 inputs2 = batch['latent_features'].float().to(device)
#                 targets = batch['target'].to(device)

#                 # zero the parameter gradients
#                 optimizer.zero_grad()
#                 # forward
#                 # track gradients if only in train
#                 with torch.set_grad_enabled(phase == 'train'):
#                     preds = model(inputs1,inputs2)
#                     loss = quantile_loss(preds, targets, quantiles)
#                     # backward + optimize only if in training phase
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                 monitor.step(loss, inputs1,inputs2, preds, targets, phase)

#             # epoch statistics
#             early_stop = monitor.log_epoch(phase)

#         if early_stop:
#             break

#         # Updates the learning rate
#         scheduler.step()

#     # load best model weights
#     model.load_state_dict(monitor.best_model_wts)
#     models.append(model)

# print(f'Training complete! Time: {timedelta(seconds=time() - t0)}')

Generate Submission CSV

In [ ]:
# data = ClinicalDataset(
#     root_dir=root_dir,
#     ctscans_dir=os.path.join(root_dir,'test'),
#     mode='test',
#     transform=transforms.Compose([
#         CropBoundingBox(),
#         ConvertToHU(),
#         Resize((40, 256, 256)),
#         Clip(bounds=clip_bounds),
#         MaskWatershed(
#             min_hu=min(clip_bounds),
#             iterations=watershed_iterations,
#             show_tqdm=False),
#         Normalize(bounds=clip_bounds),
#         ToTensor(),
#          ZeroCenter(pre_calculated_mean=pre_calculated_mean)
#     ]))

# data.cache(latent_dir)                         

In [ ]:
autoencoder = AutoEncoder()
autoencoder.load_state_dict(torch.load(
    pretrained_ae_weigths,
    map_location=torch.device('cuda')
))
# print(pretrained_ae_weigths.shape)
autoencoder.to(device)
autoencoder.eval()

data = ClinicalDataset(
    root_dir=root_dir,
    ctscans_dir=os.path.join(root_dir,'test'),
    cache_dir="/kaggle/working/test_cache",
    mode='test',
    transform=GenerateLatentFeatures(autoencoder,"/kaggle/working/test_cache","/kaggle/working/test_latent",'test')
)
for i in trange(len(data)):
    sample = data[i]

In [ ]:
# raw=check_submit()

# avg_preds = np.zeros((len(data), len(quantiles)))

# df = pd.DataFrame(data=avg_preds, columns=list(quantiles))
# df['Patient_Week'] = raw['Patient_Week']
# df['FVC'] = 5000.0
# df['Confidence'] = 500.0
# df = df.drop(columns=list(quantiles))
# df.to_csv('submission.csv', index=False)

In [ ]:
# data = ClinicalDataset(
#     root_dir=root_dir,
#     ctscans_dir=os.path.join(root_dir,'test'),
#     cache_dir=latent_dir,
#     mode='test',
#     transform=GenerateLatentFeatures(autoencoder, latent_dir)
# )

# data = ClinicalDataset(
#     root_dir=root_dir,
#     ctscans_dir=os.path.join(root_dir,'test'),
#     cache_dir="/kaggle/working/test_cache",
#     mode='test',
#     transform=GenerateLatentFeatures(autoencoder,'/kaggle/working/test_latent')
# )

autoencoder = AutoEncoder()
autoencoder.load_state_dict(torch.load(
    pretrained_ae_weigths,
    map_location=torch.device('cuda')
))
# print(pretrained_ae_weigths.shape)
autoencoder.to(device)
autoencoder.eval()

data = ClinicalDataset(
    root_dir=root_dir,
    ctscans_dir=os.path.join(root_dir,'test'),
    cache_dir="/kaggle/working/test_cache",
    mode='test',
    transform=GenerateLatentFeatures(autoencoder,"/kaggle/working/test_cache","/kaggle/working/test_latent",'test')
)

avg_preds = np.zeros((len(data), len(quantiles)))

for model in models:
    dataloader = DataLoader(data, batch_size=batch_size,
                            shuffle=False, num_workers=0)
    preds = []
    for batch in tqdm(dataloader):
#         inputs1 = batch['tabular_features'].float().to(device)
#         inputs2 = batch['latent_features'].float().to(device)
        inputs1 = batch['tabular_features'].float().to(device)
        inputs2 = batch['latent_features'].float().to(device)
#         targets = batch['target'].to(device)
        with torch.no_grad():
            out=model(inputs1, inputs2).cpu()
            preds.append(out)
    preds = torch.cat(preds, dim=0).numpy()
#     preds=preds.numpy()
    avg_preds += preds

avg_preds /= len(models)
df = pd.DataFrame(data=avg_preds, columns=list(quantiles))
df['Patient_Week'] = data.raw['Patient_Week']
df['FVC'] = df[quantiles[1]]
df['Confidence'] = df[quantiles[2]] - df[quantiles[0]]
df = df.drop(columns=list(quantiles))
df.to_csv('submission.csv', index=False)


In [ ]:
df